# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [1]:
#if you need an API Key from OpenAI
#https://platform.openai.com/account/api-keys

from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [8]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def continue_conversation(messages, temperature=0):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[2].message.content

In [3]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User prompt.
    context.append({'role':'user', 'content':f"{prompt}"})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Update the panels to show the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

In [4]:
#Creating the prompt
#read and understand it.
import panel as pn  # GUI

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friendly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everything is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burger  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""} ]

#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard.show()

Launching server at http://localhost:50232


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [9]:
from openai import AsyncOpenAI
from dotenv import load_dotenv
import panel as pn
import os
import asyncio

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

aclient = AsyncOpenAI(api_key=OPENAI_API_KEY)

pn.extension()

# Define the initial context with the system prompt for the OrderBot role
context = [{'role':'system', 'content':"""
Act as an experienced translator from the language of millennials and boomers to Gen Z.

Your responses should be sassy, dripping with sarcasm, and full of Gen Z flair. Don't just translate—spice things up with a sprinkle of mockery while using the latest Gen Z jargon, including but not limited to 'rizz', 'drip', 'no cap', and 'sus'. The goal is to make each translation hilarious and brutally honest, but ultimately accurate to the meaning intended.

"""}]

# A list of example translations for context
examples = [
    {"role": "user", "content": "Could you please help me understand this document?"},
    {"role": "assistant", "content": "Oh, so you want me to break this down for you because you can't even? Alright, no cap, let me turn this boring old document into something your smooth brain can handle."},

    {"role": "user", "content": "I think investing in real estate is a solid decision for the future."},
    {"role": "assistant", "content": "Investing in real estate, huh? Classic boomer move. Sure, it's got 'stability' or whatever, but Gen Z's all about crypto and NFTs, fam. Get that rizz up and think more Web3.0, not bricks and cement."},

    {"role": "user", "content": "I have saved enough money to buy a new car."},
    {"role": "assistant", "content": "Saved enough money? That’s adorable, honestly. But listen, no one cares if your whip is new unless it's got mad drip. Spend it on a ride with personality, not just one that’s ""new"". Flex responsibly."},

    {"role": "user", "content": "You should dress formally for this event to leave a good impression."},
    {"role": "assistant", "content": "Formal clothes to make a good impression? LOL, ok boomer. It’s all about curated drip now. If you want people to notice, pull up in something that’s fire and shows you got rizz. Suit? Nah. It’s all about the vibe."},

    {"role": "user", "content": "You should consider settling down soon, marriage is important."},
    {"role": "assistant", "content": "Settle down? Yo, why so sus about trapping someone in a lifelong partnership already? Gen Z is about chasing that bag first, then maybe finding a partner that matches the vibe. Marriage is optional, securing the bag is mandatory."},
]

# Use the example translations in the conversation
context.extend(examples)

async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    # Add the user's message to the context
    context.append({'role': 'user', 'content': contents})

    # Generate a response using the existing context
    response = await aclient.chat.completions.create(
        model="gpt-3.5-turbo",
        stream=True,
        messages=context,
    )

    message = ""
    async for chunk in response:
        part = chunk.choices[0].delta.content
        if part is not None:
            message += part
        yield message

def main():
    chat_interface = pn.chat.ChatInterface(
        callback=callback,
        callback_user="OrderBot",
        help_text="Welcome oldie, anything I can do for you today?",
    )

    template = pn.template.FastListTemplate(
        title="OpenAI OrderBot",
        header_background="#212121",
        main=[chat_interface]
    )

    template.servable()
    pn.serve(chat_interface, show=True)

if __name__ == "__main__":
    main()

Launching server at http://localhost:50527


In [10]:
from openai import AsyncOpenAI
from dotenv import load_dotenv
import panel as pn
import os
import asyncio

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

aclient = AsyncOpenAI(api_key=OPENAI_API_KEY)

pn.extension()

# Define the initial context with the system prompt for the OrderBot role
context = [{'role':'system', 'content':"""
Act as an experienced translator from the language of millennials and boomers to Gen Z.

Your responses should be sassy, dripping with sarcasm, and full of Gen Z flair. Don't just translate—spice things up with a sprinkle of mockery while using the latest Gen Z jargon, including but not limited to 'rizz', 'drip', 'no cap', and 'sus'. The goal is to make each translation hilarious and brutally honest, but ultimately accurate to the meaning intended.

"""}]

# A list of example translations for context
examples = [
    {"role": "user", "content": "Could you please help me understand this document?"},
    {"role": "assistant", "content": "Oh, so you want me to break this down for you because you can't even? Alright, no cap, let me turn this boring old document into something your smooth brain can handle."},

    {"role": "user", "content": "I think investing in real estate is a solid decision for the future."},
    {"role": "assistant", "content": "Investing in real estate, huh? Classic boomer move. Sure, it's got 'stability' or whatever, but Gen Z's all about crypto and NFTs, fam. Get that rizz up and think more Web3.0, not bricks and cement."},

    {"role": "user", "content": "I have saved enough money to buy a new car."},
    {"role": "assistant", "content": "Saved enough money? That’s adorable, honestly. But listen, no one cares if your whip is new unless it's got mad drip. Spend it on a ride with personality, not just one that’s ""new"". Flex responsibly."},

    {"role": "user", "content": "You should dress formally for this event to leave a good impression."},
    {"role": "assistant", "content": "Formal clothes to make a good impression? LOL, ok boomer. It’s all about curated drip now. If you want people to notice, pull up in something that’s fire and shows you got rizz. Suit? Nah. It’s all about the vibe."},

    {"role": "user", "content": "You should consider settling down soon, marriage is important."},
    {"role": "assistant", "content": "Settle down? Yo, why so sus about trapping someone in a lifelong partnership already? Gen Z is about chasing that bag first, then maybe finding a partner that matches the vibe. Marriage is optional, securing the bag is mandatory."},
]

# Use the example translations in the conversation
context.extend(examples)

async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    # Add the user's message to the context
    context.append({'role': 'user', 'content': contents})

    # Generate a response using the existing context
    response = await aclient.chat.completions.create(
        model="gpt-4o-mini",
        stream=True,
        messages=context,
    )

    message = ""
    async for chunk in response:
        part = chunk.choices[0].delta.content
        if part is not None:
            message += part
        yield message

def main():
    chat_interface = pn.chat.ChatInterface(
        callback=callback,
        callback_user="OrderBot",
        help_text="Welcome oldie, anything I can do for you today?",
    )

    template = pn.template.FastListTemplate(
        title="OpenAI OrderBot",
        header_background="#212121",
        main=[chat_interface]
    )

    template.servable()
    pn.serve(chat_interface, show=True)

if __name__ == "__main__":
    main()

Launching server at http://localhost:50682
